In [ ]:
!pip install kaggle

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data collection : kaggle API

In [ ]:
kaggle_dictionnary = json.load(open('kaggle.json'))
print(kaggle_dictionnary)
kaggle_dictionnary.keys()


{'username': 'salmabenhssin', 'key': 'ba33c8df35219bcd61c9826dcb51d3f2'}


dict_keys(['username', 'key'])

In [ ]:
#setup kaggle credentials(api) as environment variables
#ce sont les variables d'environ: 'KAGGLE_USERNAME,KAGGLE_KEY'
os.environ['KAGGLE_USERNAME'] = kaggle_dictionnary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionnary['key']

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [ ]:
#unzip the dataset file

with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


loading the dataset


In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
#convertit sentiment en val numerique tq 1 postive , en creant un dict contenant autre dict
data.replace({"sentiment" : {'positive':1,'negative':0}},inplace=True)
#inplace : impose de remplacer ca ds dataframe !!



In [ ]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
#spli data into train data and test data
train_data , test_data = train_test_split(data,test_size=0.2, random_state=42)

In [ ]:
print(train_data.shape)
print(test_data.shape)


(40000, 2)
(10000, 2)


data preprocessing

In [ ]:
#Tokenize text data pour que model undrestand our data :Xtrain & test
#tokenize convertit a word into integer tq ici il va prendre 50000 commons words in dataset

tokenizer = Tokenizer(num_words=5000)
#we fit this tokenize oncolonne qu on veut ENCODER !!
tokenizer.fit_on_texts(train_data['review'])
#puis on va coder jomal texts into sequence (liste) de meme long max:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [ ]:
print(X_train)
print(X_test)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [ ]:
print(Y_train)
print(Y_test)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


LSTM - Long Short-term Memory : maintain the sequence of input data cad reccurent :** it has the memory of others precedents words** so** it try to undrestand a sequence of words not chaque word seperately **

In [ ]:
#build the model
model = Sequential()
#this is the input layer
#5000 car 5000 words in tokenizer ecrite et 200 est taille de sequence mentionée en pad seq
#Construction du modèle LSTM : Couche d'Embedding : La couche d'embedding transforme les séquences de nombres entiers en vecteurs denses de taille fixe, ce qui permet au modèle de capturer les relations entre les mots.
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
#second layer
#rappel unit === neurone
#dropout indice que model nest pas en overfitting :purpose of dropout is verifying that the model is not overfitting
model.add(LSTM(units=128,dropout=0.2,recurrent_dropout=0.2))
#outputlayer
model.add(Dense(units=1,activation='sigmoid'))





/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(X_train,Y_train,batch_size=64,epochs=5,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 279ms/step - accuracy: 0.7030 - loss: 0.5538 - val_accuracy: 0.8440 - val_loss: 0.3614
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 138s 272ms/step - accuracy: 0.8457 - loss: 0.3658 - val_accuracy: 0.8342 - val_loss: 0.3865
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 128s 257ms/step - accuracy: 0.8718 - loss: 0.3191 - val_accuracy: 0.8410 - val_loss: 0.3609
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 144s 261ms/step - accuracy: 0.8908 - loss: 0.2780 - val_accuracy: 0.8566 - val_loss: 0.3468
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 147s 272ms/step - accuracy: 0.9062 - loss: 0.2472 - val_accuracy: 0.8739 - val_loss: 0.3244


Model evaluation

In [ ]:
loss,accuracy = model.evaluate(X_test,Y_test)
print(f"loss : {loss}")
print(f"accuracy : {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 83ms/step - accuracy: 0.8690 - loss: 0.3209
loss : 0.31384196877479553
accuracy : 0.8716999888420105


Build a Predictive system

In [ ]:
def predict_sentiment(review):
  #firststep is tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence,maxlen=200)
  prediction = model.predict(padded_sequence)
  if prediction[0][0] > 0.5:
    return "positive"

  else:
    return "negative"

In [ ]:
new_review= "this movie was fantastic . I loved it ."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
The sentiment of the review is : positive


In [ ]:
new_review= "this movie was not good ."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
The sentiment of the review is : negative
